In [1]:
# 使用

In [1]:
import keras

Using TensorFlow backend.


In [2]:
import tensorflow as tf

In [3]:
keras.__version__

'2.2.4'

In [4]:
tf.__version__

'1.12.0'

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error

In [7]:
train = pd.read_csv("../../kaggle/LosAngelesHousePricesForecast/data/train_1.csv")

In [8]:
train.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SqrtLotArea
0,0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,...,None,None,None,0,2,2008,WD,Normal,208500,91.923882
1,1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,...,None,None,None,0,5,2007,WD,Normal,181500,97.979590
2,2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,...,None,None,None,0,9,2008,WD,Normal,223500,106.066017
3,3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,...,None,None,None,0,2,2006,WD,Abnorml,140000,97.724101
4,4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,...,None,None,None,0,12,2008,WD,Normal,250000,119.415242


In [9]:
if 'Unnamed: 0' in train.columns: 
    train = train.drop(['Unnamed: 0'], axis=1)

In [10]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SqrtLotArea
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0,2,2008,WD,Normal,208500,91.923882
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0,5,2007,WD,Normal,181500,97.979590
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,9,2008,WD,Normal,223500,106.066017
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,2,2006,WD,Abnorml,140000,97.724101
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,12,2008,WD,Normal,250000,119.415242


In [11]:
y=np.asarray(train['SalePrice'])
train1=train.drop(['Id','SalePrice'],axis=1)
X=np.asarray(pd.get_dummies(train1).reset_index(drop=True))
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.2)

In [12]:
def benchmark(model):
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    logrmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(pred)))
    print("RMSE: {} \nLOGRMSE: {}".format(rmse, logrmse))
    return rmse, logrmse

In [13]:
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(30,activation="relu"))  
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [14]:
model = create_model()

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                3040      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                330       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 3,401
Trainable params: 3,401
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0)

In [17]:
benchmark(model)

RMSE: 57626.14836654586 
LOGRMSE: 0.23028409875704253


(57626.14836654586, 0.23028409875704253)

In [18]:
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(300,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [19]:
model = create_model()

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 10)                3040      
_________________________________________________________________
dense_5 (Dense)              (None, 300)               3300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                15050     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 21,441
Trainable params: 21,441
Non-trainable params: 0
_________________________________________________________________


In [21]:
history = model.fit(X_train, y_train, epochs=2020, batch_size=32, verbose=0)

In [22]:
benchmark(model)

RMSE: 43665.49252915732 
LOGRMSE: 0.17321879872313092


(43665.49252915732, 0.17321879872313092)